# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download('punkt')
import pandas as pd
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet') # download for lemmatization
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, fbeta_score, recall_score, classification_report, accuracy_score, precision_score, make_scorer, precision_recall_fscore_support
from sklearn.model_selection import GridSearchCV
from workspace_utils import active_session
import pickle
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_data.db')
df = pd.read_sql("SELECT * FROM disaster_data",engine)
# df.head()
X = df.message.values
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]"," ",text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,Y)

In [6]:
# Take only 10% of data for fitting using GridSearchCV so that it's faster
# train = np.concatenate((X_train, y_train), axis=1)
# X_train_d = X_train[:,np.newaxis]
# train = np.hstack((X_train, y_train))

# train_sample = train.sample(frac=0.1, replace=False, random_state=None, axis=0)

number_of_rows = X_train.shape[0]

# np.random.seed(123) # uncomment if we want repeatable results
random_rows = np.random.choice(number_of_rows, size=int(0.10*number_of_rows), replace=False)

X_train_sample = X_train[random_rows] # this is numpy ndarray
y_train_sample = y_train.iloc[random_rows,:] # this is pandas df

In [35]:
# pipeline.fit(X_train_sample,y_train_sample) # for now, do only with a subset, to compare to tuned model, which needs subset to run in reasonable time

pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [36]:
predicted = pipeline.predict(X_test)

In [37]:
# predicted.shape

In [38]:
# y_test.iloc[:,0].value_counts()

In [39]:
# f1_score = fbeta_score(y_test,predicted,beta=1)
# precision = fbeta_score(y_test,predicted,beta=0)
# recall =  recall_score(y_test,predicted)

# for i in range(y_test.shape[1]):
#     print(classification_report(y_test.iloc[:,i],predicted[:,i]))

# print(classification_report(y_test.values,predicted,target_names=Y.columns.values))

category_names = Y.columns.values
for i, c in enumerate(category_names): 
    print(c)
#     if i==1:
#         test=classification_report(y_test.iloc[:,i], predicted[:,i],output_dict=True)
    print(classification_report(y_test.iloc[:,i], predicted[:,i])) # the averages given here are weighted
    print('Accuracy: ', accuracy_score(y_test.iloc[:,i],predicted[:,i]), '\n')


related
             precision    recall  f1-score   support

          0       0.62      0.45      0.52      1557
          1       0.84      0.91      0.88      4950

avg / total       0.79      0.80      0.79      6507

Accuracy:  0.802213001383 

request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5399
          1       0.81      0.45      0.58      1108

avg / total       0.88      0.89      0.88      6507

Accuracy:  0.888735208237 

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6476
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6507

Accuracy:  0.9952358998 

aid_related
             precision    recall  f1-score   support

          0       0.76      0.85      0.80      3784
          1       0.75      0.62      0.68      2723

avg / total       0.75      0.75      0.75      6507

Accuracy:  0.75226678961

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [40]:
metrics = dict()
for i, c in enumerate(category_names): 
    print(c)
    metrics[c] = precision_recall_fscore_support(y_test.iloc[:,i], predicted[:,i],average='weighted')
    print(metrics[c],'\n')
#     print('Accuracy: ', accuracy_score(y_test.iloc[:,i],predicted[:,i]), '\n')

related
(0.7877008798926457, 0.8022130013831259, 0.79058655121166244, None) 

request
(0.88248752420835974, 0.88873520823728291, 0.87506156967090343, None) 

offer
(0.99049449625114405, 0.99523589980021521, 0.99285953741141386, None) 

aid_related
(0.75159316004825505, 0.75226678961118798, 0.74786075388464002, None) 

medical_help
(0.9015628059202655, 0.92454280006147227, 0.89823411170113043, None) 

medical_products
(0.9425595862825672, 0.95143691409251574, 0.93381853238229007, None) 

search_and_rescue
(0.96176286055676807, 0.97279852466574457, 0.96122420385687368, None) 

security
(0.96313822651487913, 0.98048255724604272, 0.97173300389124373, None) 

military
(0.95319520573285765, 0.96419240817581064, 0.95020877379628366, None) 

child_alone
(1.0, 1.0, 1.0, None) 

water
(0.94977917282276858, 0.9528200399569694, 0.94185757365701073, None) 

food
(0.92634874184573901, 0.93022898417089284, 0.92098668949839779, None) 

shelter
(0.93236575116428377, 0.93822037805440295, 0.9266213622525

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


 

buildings
(0.94411203973282209, 0.95451052712463502, 0.93752301593770859, None) 

electricity
(0.97374930697732609, 0.97986783463961891, 0.97077005891292012, None) 

tools
(0.98621656728438634, 0.9930843706777317, 0.98963855398558132, None) 

hospitals
(0.97920863982373219, 0.98954971569079453, 0.98435201905345693, None) 

shops
(0.98988279671578283, 0.99492853849700325, 0.99239925402197038, None) 

aid_centers
(0.97434829732253458, 0.98709082526509917, 0.98067817025177595, None) 

other_infrastructure
(0.93136311475717926, 0.95727677885354234, 0.93727717603729865, None) 

weather_related
(0.86633188210705658, 0.86844936222529578, 0.8620881644373003, None) 

floods
(0.93521097370054551, 0.9400645458736745, 0.92794475890933859, None) 

storm
(0.92825974089355645, 0.93576148762870759, 0.92737985861441186, None) 

fire
(0.99011057328789309, 0.99001075764561242, 0.98533075893530453, None) 

earthquake
(0.9630679677956645, 0.96465345013062853, 0.96273894809298688, None) 

cold
(0.9789850

In [41]:
# Test

metrics['related'][2] # f score of 'related' column

0.79058655121166244

In [42]:
# This is cleaner (but only gives values for positive class)
# print(classification_report(y_test, predicted, target_names=Y.columns))

In [43]:
# The following doesn't give averages either
category_names = Y.columns.values
def get_metrics_summary(y_test,y_pred):
    metrics_summary = pd.DataFrame(index = category_names, columns = ['accuracy', 'precision', 'recall', 'f-1_score'])
    for i, c in enumerate(category_names): 
        metrics_summary.loc[c,'accuracy'] = accuracy_score(y_test.iloc[:,i],y_pred[:,i])
        metrics_summary.loc[c,'precision'] = precision_score(y_test.iloc[:,i],y_pred[:,i])
        metrics_summary.loc[c,'recall'] = recall_score(y_test.iloc[:,i],y_pred[:,i])
        metrics_summary.loc[c,'f-1_score'] = fbeta_score(y_test.iloc[:,i],y_pred[:,i],beta=1)

    metrics_summary.loc['average'] = metrics_summary.mean(axis=0)
    
    return metrics_summary

In [44]:
# metrics_summary_1 = get_metrics_summary(y_test,predicted)
# metrics_summary_1

### 6. Improve your model
Use grid search to find better parameters. 

In [45]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fc4ee2a1f28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [ ]:
# parameters = {
#     'clf__estimator__max_depth':[2],
#     'clf__estimator__n_estimators':[20,50]
#     }

# parameters = {
#     'vect__max_df': (0.5, 0.75, 1.0),
#     'vect__ngram_range': ((1, 1), (1,2)),
#     'vect__max_features': (None, 5000,10000),
#     'tfidf__use_idf': (True, False)
# }

# parameters = {
#     'vect__ngram_range': ((1, 1), (1, 2)),
#     'clf__estimator__bootstrap': (True, False)
# }

# parameters = { # try next
#     'clf__estimator__n_estimators': [50, 100, 150],
#     'clf__estimator__min_samples_split': [2, 3, 4]
# }

# parameters = { # try
#         'clf__estimator__n_estimators': [100, 200],
#         'clf__estimator__learning_rate': [0.1, 0.3]
#     }

parameters = { # try next
    'clf__estimator__n_estimators': [200,300],
    'clf__estimator__min_samples_split': [2,3]
}

# BEST PARAMS for [50,100] and [2,3] was {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}****
# BEST PARAMS for [100,200] and [2,3] was {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 100}

# parameters = { # try next
# #     'clf__estimator__criterion': ['gini', 'entropy'], # In this, keep gini or entropy.
#     'clf__estimator__max_depth': [2, 5], # Use only two [2,5,None]
#     'clf__estimator__n_estimators': [100, 200], # Use only two [10,20,50]
#     'clf__estimator__min_samples_leaf':[2, 5], # can be ignored [1,5,10]
# }

# scoring = {'accuracy': make_scorer(accuracy_score), 'precision': make_scorer(precision_score), 'recall': make_scorer(recall_score)}

# cv = GridSearchCV(pipeline,param_grid=parameters,scoring=scoring,refit='accuracy') #If scoring not included, refit not needed and grid search would find best model based on estimator's score method, which is average accuracy

# cv = GridSearchCV(pipeline,param_grid=parameters)

scoring = make_scorer(f1_score, average='weighted')
# cv = GridSearchCV(pipeline,param_grid=parameters, n_jobs=-1, verbose=2, scoring = ‘f1_weighted’) # the higher the verbose the more information

# cv = GridSearchCV(pipeline,param_grid=parameters, n_jobs=-1, verbose=2, scoring = scoring)
cv = GridSearchCV(pipeline,param_grid=parameters, n_jobs=-1, verbose=2) # better results when leave default scoring

with active_session():
    cv.fit(X_train,y_train)
#     cv.fit(X_train_sample,y_train_sample)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200, total=12.6min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 13.8min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200, total=12.7min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200, total=12.9min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=300 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=300, total=19.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=300 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=300, total=19.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=300 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=300, total=19.1min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=200 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=200, total=11.3min
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=200

In [ ]:
y_pred_improved = cv.predict(X_test)

In [ ]:
# import pickle

# with open('train_classifier.pkl', 'wb') as file:
#     pickle.dump(cv, file)

In [ ]:
# with open('train_classifier.pkl', "rb") as input_file:
#     e = pickle.load(input_file)

In [ ]:
# y_pred_best = e.predict(X_test)
# e.best_params_

In [ ]:
# e.get_params()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# print(classification_report(y_test, y_pred_improved, target_names=Y.columns))

In [ ]:
# metrics_2 = get_metrics_summary(y_test,y_pred_improved)
# metrics_2

In [ ]:
# **COMPLETE THIS FUNCTION**

def compare_metric(y_test,y_pred1,y_pred2,metric='f-1_score'):
    metrics_summary = pd.DataFrame(index = category_names, columns = ['metric_old', 'metric_new'])
    for i, c in enumerate(category_names):
        if metric=='accuracy':
#             metrics_summary.loc[c,'metric_old'] = accuracy_score(y_test.iloc[:,i],y_pred1[:,i])
#             metrics_summary.loc[c,'metric_new'] = accuracy_score(y_test.iloc[:,i],y_pred2[:,i])
            None
        elif metric=='precision':
#             metrics_summary.loc[c,'metric_old'] = precision_score(y_test.iloc[:,i],y_pred1[:,i])
#             metrics_summary.loc[c,'metric_new'] = precision_score(y_test.iloc[:,i],y_pred2[:,i])
            None
        elif metric=='recall':
#             metrics_summary.loc[c,'metric_old'] = recall_score(y_test.iloc[:,i],y_pred1[:,i])
#             metrics_summary.loc[c,'metric_new'] = recall_score(y_test.iloc[:,i],y_pred2[:,i])
            None
        elif metric=='f-1_score':
#             metrics_summary.loc[c,'metric_old'] = fbeta_score(y_test.iloc[:,i],y_pred1[:,i],beta=1)
#             metrics_summary.loc[c,'metric_new'] = fbeta_score(y_test.iloc[:,i],y_pred2[:,i],beta=1)
            metrics_summary.loc[c,'metric_old'] = precision_recall_fscore_support(y_test.iloc[:,i], y_pred1[:,i], average='weighted')[2] # 2 for 3rd entry in tuple for f-score
            metrics_summary.loc[c,'metric_new'] = precision_recall_fscore_support(y_test.iloc[:,i], y_pred2[:,i], average='weighted')[2]
        
    metrics_summary['improved'] = metrics_summary['metric_new']>=metrics_summary['metric_old']
    metrics_summary.loc['sum'] = metrics_summary.sum()

#     metrics_summary.loc['average'] = metrics_summary.mean(axis=0)
    
    return metrics_summary

In [ ]:
metrics_comparison = compare_metric(y_test,predicted,y_pred_improved)
metrics_comparison

In [ ]:
cv.best_params_

# 0.948995	0.592643	0.208286	0.258425 with n_estimators = 200 and min_samples_split =2 as best from [100,200] and [2,3]

In [ ]:
a=1
with active_session():
    while a==1:
        b=2

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# Try using different classifier - LEFT OFF HERE RUNNING***

pipeline2 = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier()))
])

In [16]:
pipeline2.fit(X_train,y_train)

predicted2 = pipeline2.predict(X_test)

In [17]:
metrics_adaboost = get_metrics_summary(y_test,predicted2)
metrics_adaboost

,accuracy,precision,recall,f-1_score
related,0.788996,0.800466,0.965088,0.875102
request,0.890733,0.78,0.517241,0.62201
offer,0.993545,0,0,0
aid_related,0.767327,0.770068,0.627726,0.69165
medical_help,0.932381,0.641129,0.311765,0.419525
medical_products,0.95743,0.617486,0.353125,0.449304
search_and_rescue,0.973874,0.62963,0.184783,0.285714
security,0.979407,0.2,0.0420168,0.0694444
military,0.97034,0.571429,0.288462,0.383387
child_alone,1,0,0,0


In [18]:
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f8d6e45bf28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [19]:
parameters2 = {
    'clf__estimator__n_estimators': [50,100],
    'clf__estimator__learning_rate': [0.5, 1.0]
}

cv2 = GridSearchCV(pipeline2,param_grid=parameters2, n_jobs=-1, verbose=2) # the higher the verbose the more information

with active_session():
    cv2.fit(X_train,y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, total= 1.5min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.6min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, total= 1.4min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, total= 1.4min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, total= 2.7min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, total= 2.7min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, total= 2.6min
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=50, total= 1.4min
[CV] clf__estimator__learning_rate=1.0, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learni

[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 27.1min finished


NameError: name 'cv' is not defined

In [20]:
y_pred2 = cv2.predict(X_test)

In [21]:
metrics_adaboost_new = get_metrics_summary(y_test,y_pred2)
metrics_adaboost_new

,accuracy,precision,recall,f-1_score
related,0.796527,0.804088,0.970907,0.879658
request,0.892116,0.830508,0.476569,0.605618
offer,0.99416,0,0,0
aid_related,0.765637,0.785576,0.6,0.680361
medical_help,0.930229,0.67284,0.213725,0.324405
medical_products,0.956816,0.656,0.25625,0.368539
search_and_rescue,0.974028,0.758621,0.119565,0.206573
security,0.981405,0.333333,0.0168067,0.032
military,0.970493,0.611111,0.211538,0.314286
child_alone,1,0,0,0


### 9. Export your model as a pickle file

In [22]:
with open('train_classifier_2.pkl', 'wb') as file:
    pickle.dump(cv2, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.